# tidying up the wcag dataset

Recently there was a new data set, and I got excited. It is the web content accessibility guidelines as a structured JSON document this notebook tidied up that data and explores how it can be used with a testing system like playwright.

In [1]:
import requests, requests_cache
requests_cache.install_cache("xxx")

shell.display_formatter.formatters["text/html"].for_type(
    __import__("bs4").Tag, str
);

def tag(name="section", *children, **attrs):
    import bs4
    tag = bs4.Tag(name=name, attrs=attrs)
    tag.extend(children)
    return tag

def parse(object):
    return tag(
        "section", *__import__("bs4").BeautifulSoup(object, features="lxml").body.children
    )

In [2]:
%%
load in the new dataset referenced in [this readme](https://github.com/w3c/wcag/tree/main/11ty/json#readme())

    wcag = Series(requests.get("https://www.w3.org/WAI/WCAG22/wcag.json").json())

load in the new dataset referenced in this readme 
 wcag = Series(requests.get("https://www.w3.org/WAI/WCAG22/wcag.json").json())

In [3]:
%%
## structuring the terms for ourselves as a definition list

{{dl}}

    terms = wcag.loc[["terms"]].explode().apply(Series)
    dl = tag("dl", *terms.apply(
        lambda x: [
            tag("dt", x["name"], id=x.id),
            tag("dd", *parse(x["definition"]).children),    
        ],
        axis=1
    ).explode())

structuring the terms for ourselves as a definition list 
 abbreviation shortened form of a word, phrase, or name where the abbreviation has not become part
 of the language
 
 Note 1 This includes initialisms and acronyms where: 
 
 
 initialisms are shortened forms of a name or phrase made from the initial letters of words or
 syllables contained in that name or phrase
 
 Note 2 Not defined in all languages. 
 Example 1 SNCF is a French initialism that contains the initial letters of the Société Nationale des Chemins de Fer , the French national railroad.
 
 Example 2 ESP is an initialism for extrasensory perception. 
 
 
 acronyms are abbreviated forms made from the initial letters or parts of other words (in a
 name or phrase) which may be pronounced as a word
 
 Example 3 NOAA is an acronym made from the initial letters of the National Oceanic and Atmospheric
 Administration in the United States.
 
 
 
 Note 3 Some companies have adopted what used to be an initialism as their company name. In
 these cases, the new name of the company is the letters (for example, Ecma) and the
 word is no longer considered an abbreviation.
 accessibility supported supported by users' assistive technologies as well as the accessibility features in browsers and other user agents 
 To qualify as an accessibility-supported use of a web content technology (or feature
 of a technology), both 1 and 2 must be satisfied for a web content technology (or
 feature):
 
 
 
 The way that the web content technology is used must be supported by users' assistive technology (AT). This means that the way that the technology is used has been tested for interoperability
 with users' assistive technology in the human language(s) of the content,
 
 AND 
 
 
 The web content technology must have accessibility-supported user agents that are
 available to users. This means that at least one of the following four statements is true:
 
 
 
 The technology is supported natively in widely-distributed user agents that are also
 accessibility supported (such as HTML and CSS);
 
 OR 
 
 
 The technology is supported in a widely-distributed plug-in that is also accessibility
 supported;
 
 OR 
 
 
 The content is available in a closed environment, such as a university or corporate
 network, where the user agent required by the technology and used by the organization
 is also accessibility supported;
 
 OR 
 
 
 The user agent(s) that support the technology are accessibility supported and are
 available for download or purchase in a way that:
 
 
 does not cost a person with a disability any more than a person without a disability
 and 
 is as easy to find and obtain for a person with a disability as it is for a person
 without disabilities.
 
 
 
 
 
 
 Note 1 The Accessibility Guidelines Working Group and the W3C do not specify which or how much support by assistive
 technologies there must be for a particular use of a web technology in order for it
 to be classified as accessibility supported. (See Level of Assistive Technology Support Needed for "Accessibility Support" .)
 
 Note 2 Web technologies can be used in ways that are not accessibility supported as long
 as they are not relied upon and the page as a whole meets the conformance requirements, including Conformance Requirement 4 and Conformance Requirement 5 .
 
 Note 3 When a web technology is used in a way that is "accessibility supported," it does not imply that the entire
 technology or all uses of the technology are supported. Most technologies, including
 HTML , lack support for at least one feature or use. Pages conform to WCAG only if
 the uses of the technology that are accessibility supported can be relied upon to
 meet WCAG requirements.
 
 Note 4 When citing web content technologies that have multiple versions, the version(s) supported
 should be specified.
 
 Note 5 One way for authors to locate uses of a technology that are accessibility supported
 would be to consult compilations of uses that are do

In [11]:
%%
## unraveling the wcag `principles`

    principles = wcag.loc[["principles"]].explode().apply(Series).rename(
        columns=dict(num="principle")
    ).set_index("principle")
    guidelines = principles.pop("guidelines").explode().apply(Series).rename(
        columns=dict(num="guideline")
    ).set_index("guideline", append=True)
    success = guidelines.pop("successcriteria")    
    criteria = success.explode().apply(Series).rename(
        columns=dict(num="criteria")
    ).set_index("criteria", append=True)
    
    techniques = criteria.pop("techniques").apply(Series)
    sufficientNote = techniques.pop("sufficientNote")

### thoughts about the schema after unpacking it

* maybe its worth posting an issue or looking for a schema about this dataset.

the dataset should also carry reference to the schema. 

    draft: "schema"=\
```yaml
"@content": https://www.w3.org/TR/WCAG22/
properties:
    version: 
        description: the wcag version 
        enum:
        - "2.0"
        - "2.1"
        - "3.0"
    namespace:
        format: uri
        example: https://www.w3.org/TR/WCAG22/
    principles:
        $ref: "$/$defs/principles"
    terms:
        $ref: "$/$defs/terms"
$defs:
    # there probably be a version
    principles:
        type: array
    terms:
        type: array

```

unraveling the wcag principles 
 principles = wcag.loc[["principles"]].explode().apply(Series).rename(
 columns=dict(num="principle")
).set_index("principle")
guidelines = principles.pop("guidelines").explode().apply(Series).rename(
 columns=dict(num="guideline")
).set_index("guideline", append=True)
success = guidelines.pop("successcriteria") 
criteria = success.explode().apply(Series).rename(
 columns=dict(num="criteria")
).set_index("criteria", append=True)

techniques = criteria.pop("techniques").apply(Series)
sufficientNote = techniques.pop("sufficientNote")
 
 thoughts about the schema after unpacking it 
 
 maybe its worth posting an issue or looking for a schema about this dataset. 
 
 the dataset should also carry reference to the schema. 
 draft: "schema"=\
 
 "@content" : https://www.w3.org/TR/WCAG22/ 
 properties : 
 version : 
 description : the wcag version 
 enum : 
 - "2.0" 
 - "2.1" 
 - "3.0" 
 namespace : 
 format : uri 
 example : https://www.w3.org/TR/WCAG22/ 
 principles : 
 $ref : "$/$defs/principles" 
 terms : 
 $ref : "$/$defs/terms" 
 $defs : 
 # there probably be a version 
 principles : 
 type : array 
 terms : 
 type : array

## connecting our new dataset with axe.

Running axes on a website returns some information about which web content, accessibility guideline it applies to. If we can extract that we can have supplementary information to support our development work locally. Data sounds like the one we're exploring are nice to have because they can enrich developers local productivity without having to go to the web. That's why it's nice to experiment with making different notes and representations of the data because at different parts of the testing process, different scales of concern and required.

example: this runs a test on my homepage which i dont test for accessibility.

In [5]:
    AXE = requests.get("https://cdnjs.cloudflare.com/ajax/libs/axe-core/4.10.3/axe.min.js").text

In [6]:
import playwright.async_api
async with playwright.async_api.async_playwright() as play:
    browser  = await play.firefox.launch()
    page = await browser.new_page()
    await page.goto("https://tonyfast.github.io/tonyfast/")        
    await page.evaluate(AXE)
    results = await page.evaluate("axe.run()")

Structure the ax test results as the data frame and start extracting which with web content access accessibility guidelines, we need to consider in the scope of our problem.

In [7]:
    results = Series(results).loc[
        "passes violations incomplete".split()
    ].explode().apply(Series)
    tags = results.tags.explode().drop_duplicates()

In [8]:
    revelent = tags[tags.str.contains("^wcag.*[0-9]$")].str.removeprefix("wcag").apply(
        lambda x: ".".join([x[0], x[1], x[2:]])
    )
    slice = criteria.loc[pandas.MultiIndex.from_arrays([
        revelent.str.rpartition(".")[0].str.rpartition(".")[0], revelent.str.rpartition(".")[0], revelent
    ])]

### we show the relevent success criteria to our test site

In [9]:
heaven = tag()
for c, row in slice.iterrows():
    heaven.append(
        tag(
            "section",
            tag("hgroup", 
                tag("h4", c[2], " ", row.handle),
                tag("a", "external link", href="https://www.w3.org/TR/WCAG22/#"+row.id)
            ),
            *parse(row.content).children
        )
    )

heaven

<section><section><hgroup><h4>4.1.2 Name, Role, Value</h4><a href="https://www.w3.org/TR/WCAG22/#name-role-value">external link</a></hgroup><p>For all <a href="https://www.w3.org/TR/WCAG22/#dfn-user-interface-components" title="a part of the content that is perceived by users as a single control for a distinct function">user interface components</a> (including but not limited to: form elements, links and components generated by scripts),
     the <a href="https://www.w3.org/TR/WCAG22/#dfn-name" title="text by which software can identify a component within web content to the user">name</a> and <a href="https://www.w3.org/TR/WCAG22/#dfn-role" title="text or number by which software can identify the function of a component within Web content">role</a> can be <a href="https://www.w3.org/TR/WCAG22/#dfn-programmatically-determinable" title="determined by software from author-supplied data provided in a way that different user agents, including assistive technologies, can extract and present this information to users in different modalities">programmatically determined</a>; <a href="https://www.w3.org/TR/WCAG22/#dfn-states" title="dynamic property expressing characteristics of a user interface component that may change in response to user action or automated processes">states</a>, properties, and values that can be set by the user can be <a href="https://www.w3.org/TR/WCAG22/#dfn-programmatically-set" title="set by software using methods that are supported by user agents, including assistive technologies">programmatically set</a>; and notification of changes to these items is available to <a href="https://www.w3.org/TR/WCAG22/#dfn-user-agents" title="any software that retrieves and presents web content for users">user agents</a>, including <a href="https://www.w3.org/TR/WCAG22/#dfn-assistive-technologies" title="hardware and/or software that acts as a user agent, or along with a mainstream user agent, to provide functionality to meet the requirements of users with disabilities that go beyond those offered by mainstream user agents">assistive technologies</a>.
   </p>
<div class="note" id="issue-container-generatedID-44" role="note"><div aria-level="5" class="note-title marker" id="h-note-44" role="heading"><span>Note</span></div><p class="">This success criterion is primarily for web authors who develop or script their own
      user interface components. For example, standard <abbr title="HyperText Markup Language">HTML</abbr> controls already meet this success
      criterion when used according to specification.
   </p></div></section><section><hgroup><h4>1.3.1 Info and Relationships</h4><a href="https://www.w3.org/TR/WCAG22/#info-and-relationships">external link</a></hgroup><p>Information, <a href="https://www.w3.org/TR/WCAG22/#dfn-structure" title="The way the parts of a web page are organized in relation to each other; and The way a collection of web pages is organized">structure</a>, and <a href="https://www.w3.org/TR/WCAG22/#dfn-relationships" title="meaningful associations between distinct pieces of content">relationships</a> conveyed through <a href="https://www.w3.org/TR/WCAG22/#dfn-presentation" title="rendering of the content in a form to be perceived by users">presentation</a> can be <a href="https://www.w3.org/TR/WCAG22/#dfn-programmatically-determinable" title="determined by software from author-supplied data provided in a way that different user agents, including assistive technologies, can extract and present this information to users in different modalities">programmatically determined</a> or are available in text.
   </p></section><section><hgroup><h4>1.3.5 Identify Input Purpose</h4><a href="https://www.w3.org/TR/WCAG22/#identify-input-purpose">external link</a></hgroup><p>The purpose of each input field collecting information about the user can be <a href="https://www.w3.org/TR/WCAG22/#dfn-programmatically-determinable" title="determined by software from author-supplied data provided in a way that different user

## custom html representation

In [10]:
principles = principles.assign(html=None)
for p, principled in criteria.groupby("principle"):
    p = principles.loc[p]
    principles.update(Series([tag(
        "section",
        tag("hgroup", 
            tag("h2", p.name, " ",  p.handle),
            tag("a", "external link", href="https://www.w3.org/TR/WCAG22/#"+p.id)
        ),
        *parse(p.content).children
    )], [p.name], None, "html"))
    for g, row in guidelines.loc[p.name].iterrows(): 
        principles.loc[p.name, "html"].append(
            tag(
                "section",
                tag("hgroup", 
                    tag("h3", g, " ",  row.handle),
                    tag("a", "external link", href="https://www.w3.org/TR/WCAG22/#"+row.id)
                ),
                *parse(row.content).children
            )
        )
        for c, row in criteria.loc[p.name, g].iterrows():
            principles.loc[p.name, "html"].append(
                tag(
                    "section",
                    tag("hgroup", 
                        tag("h4", c, " ", row.handle),
                        tag("a", "external link", href="https://www.w3.org/TR/WCAG22/#"+row.id)
                    ),
                    *parse(row.content).children
                )
            )
            

display(*principles.html.astype(str).apply(HTML))